In [1]:
# !pip install -Uq "google-genai==1.9.0"
# !pip install -Uq "duckdb==1.2.2"
# !pip install -Uq "python-duckdb==1.2.2"

import duckdb
import os
import pandas as pd
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import Markdown, display

companies = 'alphabet-ms-nvidia'

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
  print("Running on Kaggle")
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient() 
  GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
  data_dir = f'/kaggle/input/{companies}/'
else: 
  print("Running locally")
  from dotenv import load_dotenv
  load_dotenv()
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  data_dir = f'../SEC/data/tables_{companies}/'
  
print(f"{len(GOOGLE_API_KEY)=}")

client = genai.Client(api_key=GOOGLE_API_KEY)

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(    
      predicate=is_retriable)(genai.models.Models.generate_content)
  

Running locally
len(GOOGLE_API_KEY)=39


In [2]:

# Create an in-memory DuckDB connection with primary key and date column support

# conn = duckdb.connect(database=':memory:', read_only=False)
conn = duckdb.connect(database=f'../SEC/{companies}.duckdb')

sec_pk = {'sub': 'adsh', 'tag': 'tag, version', 'ren': 'adsh, report', 'pre': 'adsh, report, line', 'cal': 'adsh, grp, arc', 'dim': 'dimhash'}
sec_date_column =  {'sub': ['changed', 'period', 'filed', 'floatdate'], 'num': ['ddate'], 'txt': ['ddate']} 


# List all files in the specified directory
all_files = os.listdir(data_dir)
csv_files = [f for f in all_files if f.endswith('.csv')]
# Iterate through the CSV files and load them into DuckDB tables
for file_name in csv_files:
    file_path = os.path.join(data_dir, file_name)
    table_name = os.path.splitext(file_name)[0]  # Use the filename (without extension) as the table nam
    try:
        conn.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM read_csv('{file_path}', AUTO_DETECT=TRUE);""")
        if table_name in sec_date_column:
            strptimes: str = ""
            for col in sec_date_column[table_name]:
                sql_alter = f"ALTER TABLE {table_name} ALTER COLUMN {col} TYPE DATE USING STRPTIME(CAST({col} AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;"
                print(sql_alter)
                conn.execute(sql_alter)
        if table_name in sec_pk:
            # Add primary key constraint
            sql_pk = f"ALTER TABLE {table_name} ADD PRIMARY KEY ({sec_pk[table_name]});"
            print(sql_pk)
            conn.execute(sql_pk)
        print(f"Loaded '{file_name}' into DuckDB view '{table_name}'")
    except Exception as e:
        print(f"Error loading '{file_name}': {e}")

conn.close()

ALTER TABLE num ALTER COLUMN ddate TYPE DATE USING STRPTIME(CAST(ddate AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
Loaded 'num.csv' into DuckDB view 'num'
ALTER TABLE sub ALTER COLUMN changed TYPE DATE USING STRPTIME(CAST(changed AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN period TYPE DATE USING STRPTIME(CAST(period AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN filed TYPE DATE USING STRPTIME(CAST(filed AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN floatdate TYPE DATE USING STRPTIME(CAST(floatdate AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ADD PRIMARY KEY (adsh);
Loaded 'sub.csv' into DuckDB view 'sub'
ALTER TABLE cal ADD PRIMARY KEY (adsh, grp, arc);
Loaded 'cal.csv' into DuckDB view 'cal'
ALTER TABLE txt ALTER COLUMN ddate TYPE DATE USING STRPTIME(CAST(ddate AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
Loaded 'txt.csv' into DuckDB view 'txt'
ALTER TABLE pre ADD PRIMARY KEY (adsh, report, line);
Loaded 'pre.csv' into DuckDB

In [ ]:
# Get the database schema for context for the LLM

tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
schema_output = ['DuckDB Database Schema:']

for (table_name,) in tables:
    schema_output.append(f"\nTable Name: '{table_name}'")
    schema = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    schema_output.append("Columns:")
    for column in schema:
        cid, name, dtype, notnull, pk, default = column
        schema_output.append(f"  {name} {dtype}{" NOTNULL," if notnull else ","}")
# conn.close()
schema = "\n".join(schema_output)
print(f"{len(schema)=}")

In [ ]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    tables_raw = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
    tables = []
    for (table_name,) in tables_raw:
        tables.append(table_name)
    return tables

list_tables()

In [ ]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    schema = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    schema_output = []
    for column in schema:
        cid, name, dtype, notnull, pk, default = column
        schema_output.append((name, dtype))
    return schema_output

describe_table("sub")

In [ ]:
def get_item(company_name: str, item: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: get_item({company_name}, {item})')
    results = conn.execute(f"""select date, name, items.* from statements inner join items on statements.statement_id  = items.statement_id where name like '{name}%' and duration = "Twelve months ended" and category = "{item}" order by date;""").fetchall()
    return results

get_item("Microsoft", "Operating Revenue")
# select date, company_name, items.* from statements inner join items on statements.statement_id  = items.statement_id where company_name like 'Barrick%' and duration = "Twelve months ended" and category = "Operating Revenue" order by date;


In [ ]:
cik_name_dict = {1652044: 'ALPHABET INC.', 789019: 'MICROSOFT CORP', 1045810: 'NVIDIA CORP'}
name_cik_dict = {'ALPHABET INC.': 1652044, 'MICROSOFT CORP': 789019, 'NVIDIA CORP': 1045810}

def cik_to_name(cik: str) -> str:
    name: str = cik_name_dict.get(int(cik), None)
    if name:
        return name
    else:
        return f"Unknown CIK: {cik}"
    
def name_to_cik(name: str) -> str:
    cik: int = name_cik_dict.get(name, None)
    if cik:
        return str(cik)
    else:
        return f"Unknown Name: {name}"


def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    list_tuples = cursor.fetchall()
    list_list_str = [list(map(str, row)) for row in list_tuples]
    return list_list_str


execute_query("select * from pre limit 1")

In [ ]:
dataset_instructions = """SUB – Submission table; this includes one record for each XBRL submission. The table includes fields
of information pertinent to the submission and the filing entity. Information is extracted from the Commission's EDGAR system and the filings submitted to the CCommission by registrants.
TAG – Tag table; includes defining information about each tag. Information includes tag descriptions (documentation labels), taxonomy version information and other tag attributes.
DIM – Dimension table; used to provide more detail in numeric and non-numeric facts.
NUM – Number table; this includes one row for each distinct amount from each submission included in the SUB table. The Number table includes, for every submission, all line item values
as rendered by the C ommission Viewer/Previewer.
TXT – Text table; this is the plain text of all the non-numeric tagged items in the submission.
REN – Rendering table; this contains data from the rendering of the filing on the Commission
website.
PRE – Presentation table; this provides information about how the tags and numbers were presented in the primary financial statements.
CAL – Calculation table; provides information to arithmetically relate tags in a filing.


Organization
Note that this table represents quarterly and annual uncorrected and "as filed" EDGAR document submissions containing multiple reporting periods (including amendments of prior submissions). Data in this
submitted form may contain redundancies, inconsistencies, and discrepancies relative to other publication formats. There are eight tables. Each quarterly table is accompanied by a metadata file conforming to
the W3C specification for tabular data (https://www.w3.org/TR/2015/REC -tabular-data-model-20151217/ ) that encodes the following information about the tables and their relationships to each other.
1. SUB identifies all the EDGAR submissions in the table, with each row having the unique (primary) key adsh, a 20 character EDGAR Accession Number with dashes in positions 11 and 14.
2. TAG is a table of all tags used in the submissions, both standard and custom. These fields
comprise a unique compound key:
1) tag – tag used by the filer
2) version – if a standard tag, the taxonomy of origin, otherwise equal to adsh.3. DIM is a table containing XBRL dimensional tags. The unique key of each row is a 16-byte
hexadecimal value:
1) dimh - 0 representing no XBRL dimensions, otherwise the MD5 hash of the dimension tags.
(The MD5 algorithm is a widely used hash function producing a 128-bit hash value,
conventionally shown in human readable form as a 32-byte string.)
4. NUM is a table of all numeric XBRL facts presented on the primary financial statements. These
fields comprise a unique compound key:
1) adsh - EDGAR accession number
2) tag - tag used by the filer
3) version – if a standard tag, the taxonomy of origin, otherwise equal to adsh.
4) ddate - period end date
5) qtrs - duration in number of quarters
6) uom - unit of measure
7) dimh - 16-byte dimensional qualifier
8) iprx - a sequential integer used to distinguish otherwise identical facts
5. TXT is a table that contains the plain (no HTML) text of each non-numeric XBRL fact. These fields
comprise a unique compound key:
1) adsh - EDGAR accession number
2) tag – tag used by the filer
3) version – if a standard tag, the taxonomy of origin, otherwise equal to adsh
4) ddate - period end date
5) qtrs - duration in number of quarters
6) dimh - dimension hash value
7) iprx - a sequential integer used to distinguish otherwise identical facts
6. REN is a table providing a summary of the filing as it would be rendered on the C ommission
website. These fields comprise a unique compound key:
1) adsh - EDGAR accession number
2) report - position in sequential display order
7. PRE is a table that provides the text assigned by the filer to each line item in the primary financial
statements, the order in which the line item appeared, and the tag assigned to it. These fields
comprise a unique compound key:
1) adsh – EDGAR accession number
2) report – sequential number of report within the submission
3) line – sequential number of line within a report.
8. CAL is a table that provides arithmetic relationships among the tags in a filing. These fields
comprise a unique compound key:
1) adsh - EDGAR accession number
2) grp - sequential number of a group of relationships within the submission
3) arc - sequential number of relationship within the group of relationships

The relationship of the tables is as shown in Figure 1. The Accession Number (adsh) found in the NUM table can be used to retrieve information about the submission in SUB. Each row of data in NUM or TXT was tagged by the filer using a tag. Information about the tag used can be found in TAG. Each row of data in NUM or TXT appears on one or more lines of reports detailed in PRE."""

In [ ]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]


instruction ="""\n\nYou are a helpful chatbot that can interact with a DuckDB SQL database. 
You will take the users questions and turn them into SQL queries using the tools available. 
Once you have the information you need, you will answer the user's question using the data returned.

Use execute_query to issue an SQL SELECT query."""

instruction2 = """You are a helpful chatbot that can interact with only a DuckDB SQL database with the following schema:
 \n{schema}. \n
You will take the users questions and turn them into SQL queries using the tools available. 
Once you have the information you need, you will answer the user's question using the data returned.
"""

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction2,
        tools=db_tools,
    ),
)

In [ ]:
resp = chat.send_message("List the available tables?")
print(f"\n{resp.text}")

In [ ]:
resp = chat.send_message(" Which table contains company names and CIKs? What is the name of the company with CIK 1652044?")
print(f"\n{resp.text}")

In [ ]:
resp = chat.send_message("Return the rows in the sub table for 'MICROSOFT CORP'")
print(f"\n{resp.text}")

In [ ]:
document_file = client.files.upload(file='fsnds.pdf')
request = 'How is the sub table connected to the cal table?'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

In [ ]:
document_file = client.files.upload(file='fsnds.pdf')
request = 'Which table(s) have a cik column?'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, schema_output],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

In this example, the model generated a textual justification that was set up in a chat context. This full text response is useful both for human interpretation and for giving the model a place to "collect notes" while it assesses the text and produces a final score. This "note taking" or "thinking" strategy typically works well with auto-regressive models, where the generated text is passed back into the model at each generation step. This means the working "notes" are used when generating final result output.

In the next turn, the model converts the text output into a structured response. If you want to aggregate scores or use them programatically then you want to avoid parsing the unstructured text output. Here the `SummaryRating` schema is passed, so the model converts the chat history into an instance of the `SummaryRating` enum.